In [ ]:
import numpy as np
import tensorflow as tf

from mnist import MNIST
import matplotlib.pyplot as plt

# Preprocessing

## Functions

In [2]:
def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation, :, :]
    shuffled_labels = labels[permutation]
    return shuffled_dataset, shuffled_labels

def one_hot_encode(np_array):
    return (np.arange(10) == np_array[:,None]).astype(np.float32)

def reformat_data(dataset, labels, image_width, image_height, image_depth):
    np_dataset_ = np.array([np.array(image_data).reshape(image_width, image_height, image_depth) for image_data in dataset])
    np_labels_ = one_hot_encode(np.array(labels, dtype=np.float32))
    np_dataset, np_labels = randomize(np_dataset_, np_labels_)
    return np_dataset, np_labels

def flatten_array(array):
    shape = array.shape
    return array.reshape([shape[0], shape[1] * shape[2] * shape[3]])

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

def flatten_tf_array (layer):
    num_features = layer.get_shape()[1:4].num_elements()
    layer_flat = tf.reshape(layer, [-1, num_features])
    return layer_flat, num_features

## MNIST Data

In [3]:
mnist_folder = './data/mnist/'
mnist_image_width = 28
mnist_image_height = 28
mnist_image_depth = 1
mnist_num_labels = 10

mndata = MNIST(mnist_folder)
# mnist_train_dataset_: list of len 60000; eachelement list of 784 len; mnist_train_labels_: list of len 60000; eachelement one int
mnist_train_dataset_, mnist_train_labels_ = mndata.load_training()
# mnist_test_dataset_: list of len 10000; eachelement list of 784 len; mnist_test_labels_: list of len 10000; eachelement one int
mnist_test_dataset_, mnist_test_labels_ = mndata.load_testing()

In [4]:
# mnist_train_dataset: (60000, 28, 28, 1); mnist_train_labels: (60000, 10)
mnist_train_dataset, mnist_train_labels = reformat_data(mnist_train_dataset_, mnist_train_labels_, mnist_image_width, mnist_image_height, mnist_image_depth)
# mnist_test_dataset: (10000, 28, 28, 1); mnist_test_labels: (10000, 10)
mnist_test_dataset, mnist_test_labels = reformat_data(mnist_test_dataset_, mnist_test_labels_, mnist_image_width, mnist_image_height, mnist_image_depth)

# Convolutional Neural Network

## Set up the parameters

In [5]:
learning_rate = 0.01
batch_size = 64
num_epochs = 20

## Defien the placeholders

In [6]:
shape_x = [None] + list(mnist_train_dataset.shape)[1:4] # batch_size, image_width, image_height, image_depth
shape_y = [None] + list(mnist_train_labels.shape)[1:4] # batch_size, num_classes

X = tf.placeholder(dtype=tf.float32, shape=shape_x)
Y = tf.placeholder(dtype=tf.float32, shape=shape_y)

## Functions for initializing new variables

In [7]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))
def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

## Function for creating a new convolutional layer

In [8]:
def new_conv_layer(input, num_input_channels, filter_size, num_filters, use_pooling):
    shape = [filter_size, filter_size, num_input_channels, num_filters]
    weights = new_weights(shape=shape)
    biases = new_biases(length=num_filters)
    layer = tf.nn.conv2d(input=input, filter=weights, strides=[1, 1, 1, 1], padding='SAME')
    layer += biases
    if use_pooling:
        layer = tf.nn.max_pool(value=layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    layer = tf.nn.relu(layer)
    return layer, weights

## Function for creating a new fully connected layer

In [9]:
def new_fc_layer(input, num_inputs, num_outputs, use_relu, use_sigmoid):
    weights = new_weights(shape=[num_inputs, num_outputs]) # number of outputs= num_neurons
    biases = new_biases(length=num_outputs)
    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)
    if use_sigmoid:
        layer = tf.nn.sigmoid(layer)
    return layer

## Create the model (graph)

In [10]:
img_layers = mnist_train_dataset.shape[-1]
filter_size_c1 = 5
filter_size_c2 = 5
num_filters_c1 = 16
num_filters_c2 = 36
n_hidden_1 = 120
n_hidden_2 = 80
num_classes = mnist_train_labels.shape[1]

# 1st convolutional layer
layer_conv1, weights_conv1 = new_conv_layer(input=X, num_input_channels=img_layers, filter_size=filter_size_c1, num_filters=num_filters_c1, use_pooling=True)
# 2nd convolutional layer
layer_conv2, weights_conv2 = new_conv_layer(input=layer_conv1, num_input_channels=num_filters_c1, filter_size=filter_size_c2, num_filters=num_filters_c2, use_pooling=True)

# 1st fully connected layer
layer_flat, num_features = flatten_tf_array (layer_conv2) # flatten the last conv layer to input to the 1st fc layer
layer_fc1 = new_fc_layer(input=layer_flat, num_inputs=num_features, num_outputs=n_hidden_1, use_relu=True, use_sigmoid=False)
# 2nd fully connected layer
layer_fc2 = new_fc_layer(input=layer_fc1, num_inputs=n_hidden_1, num_outputs=n_hidden_2, use_relu=True, use_sigmoid=False)
#last fully connected layer (output layer)
layer_fc3 = new_fc_layer(input=layer_fc2, num_inputs=n_hidden_2, num_outputs=num_classes, use_relu=False, use_sigmoid=True)

## Set up operations

In [11]:
logits = layer_fc3
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
# optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

y_pred = tf.nn.softmax(logits)

init = tf.global_variables_initializer()

## Run the session

In [12]:
with tf.Session() as sess:
    sess.run(init)
    total_batch_train = int(mnist_train_dataset.shape[0] / batch_size) # 60000/batch_size: number of batches for train dataset
    
    for iter_num in range(num_epochs):
        avg_train_acc = 0.
        for i in range(total_batch_train):
            train_x = mnist_train_dataset[(i) * batch_size: (i + 1) * batch_size, ...] # (batch_size, 28, 28, 1)
            train_y = mnist_train_labels[(i) * batch_size: (i + 1) * batch_size, ...] # (batch_size, 10)
            feed_dict = {X: train_x, Y: train_y}
            _, train_pred = sess.run([optimizer, y_pred], feed_dict=feed_dict) #train_pred:(batch_size, 10)
            train_accuracy = accuracy(train_pred, train_y)
            avg_train_acc += train_accuracy
            
        test_x = mnist_test_dataset # (10000, 28, 28, 1)
        test_y = mnist_test_labels # (1000, 10)
        feed_dict = {X: test_x, Y: test_y}
        test_pred = sess.run(y_pred, feed_dict=feed_dict)
        test_accuracy = accuracy(test_pred, test_y)
        
#         print the train and test accuracies
        print("Train acc: %f, Test_acc: %f" % (avg_train_acc/total_batch_train, test_accuracy))

Train acc: 89.456043, Test_acc: 96.000000
Train acc: 96.177962, Test_acc: 97.320000
Train acc: 97.088447, Test_acc: 97.650000
Train acc: 97.573706, Test_acc: 97.980000
Train acc: 97.903882, Test_acc: 98.090000
Train acc: 98.075640, Test_acc: 98.220000
Train acc: 98.272412, Test_acc: 98.190000
Train acc: 98.374133, Test_acc: 98.250000
Train acc: 98.489194, Test_acc: 98.390000
Train acc: 98.562567, Test_acc: 98.440000
Train acc: 98.615928, Test_acc: 98.430000
Train acc: 98.702641, Test_acc: 98.500000
Train acc: 98.730990, Test_acc: 98.670000
Train acc: 98.821038, Test_acc: 98.620000
Train acc: 98.836046, Test_acc: 98.540000
Train acc: 98.894410, Test_acc: 98.820000
Train acc: 98.959445, Test_acc: 98.750000
Train acc: 99.001134, Test_acc: 98.640000
Train acc: 99.021145, Test_acc: 98.720000
Train acc: 99.067836, Test_acc: 98.640000
